In [0]:
!wget -qq https://raw.githubusercontent.com/svinkapeppa/trash/master/conlleval.py
!wget -qq https://raw.githubusercontent.com/svinkapeppa/trash/master/utils.py
!wget -qq https://raw.githubusercontent.com/svinkapeppa/trash/master/install
!pip install git+https://www.github.com/keras-team/keras-contrib.git

In [0]:
!chmod 777 install
!./install

In [3]:
from keras.models import Model
from keras.layers import TimeDistributed, Conv1D, Dense, Embedding, Input, Dropout, LSTM, Bidirectional, MaxPooling1D, Flatten, concatenate
from keras.initializers import RandomUniform
from keras.optimizers import SGD
from keras.losses import sparse_categorical_crossentropy
from keras_contrib.layers import CRF
from keras_contrib.losses import crf_loss

import utils
import random
import time
import math
import pickle

Using TensorFlow backend.


In [0]:
NUM_EPOCHS = 10000
BATCH_SIZE = 32
LSTM_DIM = 300
CHAR_EMB_DIM = 25
KERNEL_SIZE = 3
CHAR_FILTERS = 25
DROPOUT = 0.5
RECURRENT_DROPOUT = 0.5

In [0]:
with open('features/all_onehot.train', 'rb') as f:
    features_train = pickle.load(f)
with open('features/all_onehot.testa', 'rb') as f:
    features_dev = pickle.load(f)
with open('features/all_onehot.testb', 'rb') as f:
    features_test = pickle.load(f)

In [0]:
with open('features/gazetteer_PERLOC.train', 'rb') as f:
    gaze_train = pickle.load(f)
with open('features/gazetteer_PERLOC.testa', 'rb') as f:
    gaze_dev = pickle.load(f)
with open('features/gazetteer_PERLOC.testb', 'rb') as f:
    gaze_test = pickle.load(f)

In [7]:
train_sentences = utils.read_sentences('data/train')
valid_sentences = utils.read_sentences('data/valid')
test_sentences = utils.read_sentences('data/test')

print('Number of TRAIN sentences: {}'.format(len(train_sentences)))
print('Number of VALID sentences: {}'.format(len(valid_sentences)))
print('Number of TEST sentences: {}'.format(len(test_sentences)))

Number of TRAIN sentences: 14041
Number of VALID sentences: 3250
Number of TEST sentences: 3453


In [0]:
utils.convert_tags(train_sentences)
utils.convert_tags(valid_sentences)
utils.convert_tags(test_sentences)

In [0]:
tag_idx, idx_tag = utils.create_tag_mapping([train_sentences, valid_sentences, test_sentences])

In [0]:
word_idx, idx_word, word_embeddings = utils.create_word_mapping('glove/glove.6B.300d.txt')

In [0]:
char_idx, idx_char = utils.create_char_mapping([train_sentences, valid_sentences, test_sentences])

In [0]:
case_idx, case_embeddings = utils.create_case_mapping()

In [0]:
max_word_length = utils.get_max_word_length([train_sentences, valid_sentences, test_sentences])

In [0]:
utils.add_auxiliary_information(train_sentences)
utils.add_auxiliary_information(valid_sentences)
utils.add_auxiliary_information(test_sentences)

In [0]:
train_batches = utils.create_batches(train_sentences, BATCH_SIZE, max_word_length, word_idx,
                                     char_idx, tag_idx, features_train, gaze_train)
valid_batches = utils.create_batches(valid_sentences, BATCH_SIZE, max_word_length, word_idx,
                                     char_idx, tag_idx, features_dev, gaze_dev)
test_batches = utils.create_batches(test_sentences, BATCH_SIZE, max_word_length, word_idx,
                                    char_idx, tag_idx, features_test, gaze_test)

In [0]:
def CharCNNBiLSTM(word_vocab_size, case_vocab_size, char_vocab_size,
                  word_embeddings_dim, case_embeddings_dim, max_word_length,
                  word_embeddings, case_embeddings, tag_set_size):
    word_input = Input(shape=(None,))
    word_embeddings = Embedding(
        word_vocab_size, word_embeddings_dim, weights=[word_embeddings], trainable=True
    )(word_input)

    case_input = Input(shape=(None,))
    case_embeddings = Embedding(
        case_vocab_size, case_embeddings_dim, weights=[case_embeddings], trainable=True
    )(case_input)
    
    char_input = Input(shape=(None, max_word_length))
    char_embeddings = TimeDistributed(Embedding(
        char_vocab_size, CHAR_EMB_DIM, embeddings_initializer=RandomUniform(minval=-math.sqrt(0.1), maxval=math.sqrt(0.1))
    ))(char_input)
    char_embeddings = Dropout(DROPOUT)(char_embeddings)
    char_embeddings = TimeDistributed(Conv1D(
        kernel_size=KERNEL_SIZE, filters=CHAR_FILTERS, padding='same', activation='tanh', strides=1
    ))(char_embeddings)
    char_embeddings = TimeDistributed(MaxPooling1D(max_word_length))(char_embeddings)
    char_embeddings = TimeDistributed(Flatten())(char_embeddings)

    feature_input = Input(shape=(None, 196))
    gaze_input = Input(shape=(None, 3))

    embeddings = concatenate([word_embeddings, case_embeddings, char_embeddings, feature_input, gaze_input])
    embeddings = Dropout(DROPOUT)(embeddings)
    
    output = Bidirectional(LSTM(
        LSTM_DIM, return_sequences=True, dropout=DROPOUT, recurrent_dropout=RECURRENT_DROPOUT
    ))(embeddings)

    lstm = TimeDistributed(Dense(LSTM_DIM, activation='elu'))(output)
    lstm = CRF(tag_set_size, sparse_target=True)(lstm)
    gaze = TimeDistributed(Dense(3, activation='elu'))(output)
    shape = TimeDistributed(Dense(151, activation='elu'))(output)
    position = TimeDistributed(Dense(45, activation='elu'))(output)
    
    return Model(inputs=[word_input, case_input, char_input, feature_input, gaze_input], outputs=[lstm, gaze, shape, position])

In [17]:
model = CharCNNBiLSTM(len(word_idx), len(case_idx), len(char_idx),
              len(word_embeddings[0]), len(case_idx), max_word_length,
              word_embeddings, case_embeddings, len(tag_idx))
model.compile(loss=[crf_loss, utils.weighted_sparse_categorical_crossentropy, sparse_categorical_crossentropy, sparse_categorical_crossentropy], optimizer=SGD(lr=0.015, decay=1e-6, momentum=0.9, clipnorm=5))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
from conlleval import evaluate


def evaluate_model(model, batches, idx_tag):
    true_seqs, pred_seqs = [], []

    for batch in batches:
        target = batch['tag']
        target = target.reshape((target.shape[0], target.shape[1]))
        logits = model.predict([batch['word'], batch['case'], batch['char'], batch['features'], batch['gaze']], verbose=False)[0].argmax(axis=-1)

        for seq_ind, seq_len in enumerate(batch['lengths']):
            true_seqs.append(' '.join([idx_tag[ind.item()] for ind in target[seq_ind, 1: seq_len + 1]]))
            pred_seqs.append(' '.join([idx_tag[ind.item()] for ind in logits[seq_ind, 1: seq_len + 1]]))

    return evaluate(true_seqs, pred_seqs, verbose=False)

In [19]:
for epoch in range(1, NUM_EPOCHS + 1):
    random.shuffle(train_batches)
    
    print('----------------------------------- EPOCH: {} -----------------------------------'.format(epoch))
    print('----------------------------------- Training -----------------------------------')
    
    start_time = time.time()

    for batch in train_batches:
        model.train_on_batch([batch['word'], batch['case'], batch['char'], batch['features'], batch['gaze']], [batch['tag'], batch['gazetteers'], batch['shape'], batch['position']])
    
    finish_time = time.time()
    
    print('Time: {:.2f}s'.format(finish_time - start_time))
    
    print('---------------------------------- Evaluating ----------------------------------')
    
    start_time = time.time()
    
    f, precision, recall = evaluate_model(model, train_batches, idx_tag)
    
    print('================================== Train Data ==================================')
    print('F1 = {:.2f}%, Precision = {:.2f}%, Recall = {:.2f}%'.format(f, precision, recall))
    
    f, precision, recall = evaluate_model(model, valid_batches, idx_tag)
    
    print('================================== Valid Data ==================================')
    print('F1 = {:.2f}%, Precision = {:.2f}%, Recall = {:.2f}%'.format(f, precision, recall))
    
    f, precision, recall = evaluate_model(model, test_batches, idx_tag)
    
    print('================================== Test  Data ==================================')
    print('F1 = {:.2f}%, Precision = {:.2f}%, Recall = {:.2f}%'.format(f, precision, recall))
    
    finish_time = time.time()
    
    print('Time: {:.2f}s\n'.format(finish_time - start_time))

----------------------------------- EPOCH: 1 -----------------------------------
----------------------------------- Training -----------------------------------
Instructions for updating:
Use tf.cast instead.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_impl.py:107: UserWarning: Converting sparse IndexedSlices to a dense Tensor with 120000600 elements. This may consume a large amount of memory.
  num_elements)


Time: 61.54s
---------------------------------- Evaluating ----------------------------------
================================== Train Data ==================================
F1 = 53.33%, Precision = 50.21%, Recall = 51.72%
================================== Valid Data ==================================
F1 = 59.68%, Precision = 52.74%, Recall = 56.00%
================================== Test  Data ==================================
F1 = 53.85%, Precision = 52.74%, Recall = 53.29%
Time: 16.41s

----------------------------------- EPOCH: 2 -----------------------------------
----------------------------------- Training -----------------------------------
Time: 44.32s
---------------------------------- Evaluating ----------------------------------
================================== Train Data ==================================
F1 = 65.35%, Precision = 69.37%, Recall = 67.30%
================================== Valid Data ==================================
F1 = 69.13%, Precision = 72.08%, Re

KeyboardInterrupt: ignored